# Pydantic with Hugging Face Models for Structured Weather Data

This tutorial demonstrates how to implement structured data validation and weather API calls with Pydantic using affordable/free Hugging Face models.

## Overview

We'll build a weather information system that:
1. Validates user location and weather requests with Pydantic
2. Generates structured weather queries
3. Performs weather API calls with validation
4. Creates final weather reports

**Key difference:** We'll use free Hugging Face models (like Mistral-7B or Llama-3-8B) via the Hugging Face Inference API instead of paid APIs.

## Setup

First, let's install the required packages.

In [ ]:
# Install required packages
!pip install pydantic huggingface_hub requests -q

In [ ]:
import os
from datetime import datetime
from typing import Optional, Literal
import json
import re
from pydantic import BaseModel, Field, field_validator, ValidationError
from huggingface_hub import InferenceClient

## Step 1: Configure Hugging Face

Get your free API token from [https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)

Then either:
- Set it as an environment variable: `export HF_TOKEN=your_token_here`
- Or paste it directly in the cell below (not recommended for shared notebooks)

In [ ]:
from google.colab import userdata

HF_TOKEN = userdata.get('hugging')

# Initialize the Hugging Face client
client = InferenceClient(token=HF_TOKEN)

# We'll use Mistral-7B-Instruct - it's free and good at following instructions
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

# Alternative free models you can try:
# "meta-llama/Meta-Llama-3-8B-Instruct"
# "microsoft/Phi-3-mini-4k-instruct"
# "HuggingFaceH4/zephyr-7b-beta"

print(f"✓ Configured to use model: {MODEL_NAME}")

## Step 2: Define Pydantic Models

These models define the structure and validation rules for our data.

In [ ]:
# Base Weather Request Model with custom validation
class WeatherRequest(BaseModel):
    name: str = Field(..., description="User's name")
    email: str = Field(..., pattern=r'^[\w\.-]+@[\w\.-]+\.\w+$', description="Valid email address")
    location: str = Field(..., min_length=2, description="City name or coordinates")
    request_type: Literal["current", "forecast", "alerts"] = Field(
        ..., description="Type of weather information requested"
    )
    
    @field_validator('location')
    @classmethod
    def validate_location(cls, v):
        # Basic validation - could be enhanced with more sophisticated location parsing
        if len(v.strip()) < 2:
            raise ValueError(f"Location must be at least 2 characters long. Got: {v}")
        return v.strip().title()

# Weather Query Model (extends WeatherRequest)
class WeatherQuery(WeatherRequest):
    query_type: Literal["current_weather", "forecast", "weather_alerts", "historical"] = Field(
        ..., description="Specific type of weather query"
    )
    time_range: Literal["now", "today", "week", "month"] = Field(
        ..., description="Time range for the weather information"
    )
    units: Literal["celsius", "fahrenheit", "kelvin"] = Field(
        default="celsius", description="Temperature units preference"
    )
    include_details: bool = Field(
        default=False, description="Whether to include detailed weather information"
    )
    tags: list[str] = Field(
        default_factory=list, description="Relevant tags for categorization"
    )

# Tool argument models
class WeatherAPIArgs(BaseModel):
    location: str = Field(..., description="City name or coordinates for weather lookup")
    query_type: Literal["current", "forecast", "alerts"] = Field(
        ..., description="Type of weather data to retrieve"
    )
    units: Literal["celsius", "fahrenheit", "kelvin"] = Field(
        default="celsius", description="Temperature units"
    )

class LocationLookupArgs(BaseModel):
    location_query: str = Field(..., description="Location name or partial name to search")
    country_filter: Optional[str] = Field(None, description="Optional country code filter")
    
    @field_validator('location_query')
    @classmethod
    def validate_location_query(cls, v):
        if len(v.strip()) < 2:
            raise ValueError(f"Location query must be at least 2 characters long. Got: {v}")
        return v.strip()

# Final output models
class WeatherData(BaseModel):
    temperature: float
    feels_like: Optional[float] = None
    humidity: Optional[int] = None
    wind_speed: Optional[float] = None
    conditions: Optional[str] = None
    timestamp: Optional[str] = None

class WeatherResponse(WeatherQuery):
    recommended_action: Literal["provide_weather_data", "suggest_locations", "request_clarification", "no_action_needed"]
    weather_data: Optional[WeatherData] = None
    location_suggestions: Optional[list[str]] = None
    additional_info: Optional[str] = None
    response_date: Optional[str] = None

print("✓ Pydantic weather models defined successfully")

## Step 3: Helper Functions for HuggingFace LLM Calls

In [ ]:
def call_hf_model(prompt: str, model_name: str = MODEL_NAME, max_tokens: int = 1000, temperature: float = 0.3):
    """
    Call Hugging Face model and return response.
    Lower temperature for more consistent structured outputs.
    """
    try:
        messages = [{"role": "user", "content": prompt}]
        
        response = client.chat_completion(
            model=model_name,
            messages=messages,
            max_tokens=max_tokens,
            temperature=temperature
        )
        
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error calling model: {e}")
        return None

def extract_json_from_response(response: str) -> str:
    """
    Extract JSON from model response that might contain additional text.
    """
    # Try to find JSON between code blocks
    if "```json" in response:
        start = response.find("```json") + 7
        end = response.find("```", start)
        return response[start:end].strip()
    elif "```" in response:
        start = response.find("```") + 3
        end = response.find("```", start)
        return response[start:end].strip()
    
    # Try to find JSON object
    try:
        start = response.find("{")
        end = response.rfind("}") + 1
        if start != -1 and end > start:
            return response[start:end]
    except:
        pass
    
    return response.strip()

print("✓ Helper functions defined")

## Step 4: Validate Weather Request

Let's test our validation with a sample weather request.

In [ ]:
def validate_weather_request(weather_request_json: str) -> Optional[WeatherRequest]:
    """
    Validate raw weather request against WeatherRequest model.
    """
    try:
        weather_request = WeatherRequest.model_validate_json(weather_request_json)
        print("✓ Weather request validated successfully")
        return weather_request
    except ValidationError as e:
        print("✗ Validation Error:")
        for error in e.errors():
            print(f"  - {error['loc'][0]}: {error['msg']}")
        return None

# Test with valid input
weather_request_json = json.dumps({
    "name": "Jane Smith",
    "email": "jane.smith@example.com",
    "location": "New York",
    "request_type": "current"
})

validated_request = validate_weather_request(weather_request_json)
if validated_request:
    print(f"\nValidated data:\n{validated_request.model_dump_json(indent=2)}")

## Step 5: Create Weather Query with LLM

Use the LLM to analyze the weather request and generate a structured WeatherQuery.

In [ ]:
def create_weather_query(validated_weather_request: WeatherRequest, max_retries: int = 3) -> Optional[WeatherQuery]:
    """
    Use LLM to analyze weather request and create a WeatherQuery with retries.
    """
    schema = WeatherQuery.model_json_schema()
    
    prompt = f"""You are a weather information AI. Analyze the following weather request and return ONLY a JSON object matching this schema:

Schema: {json.dumps(schema, indent=2)}

Weather Request:
{validated_weather_request.model_dump_json(indent=2)}

Requirements:
- Return ONLY valid JSON, no additional text
- Include all required fields
- Choose appropriate query_type, time_range, and units
- Add relevant tags (e.g., ["temperature", "humidity", "forecast"])

JSON Response:"""

    for attempt in range(max_retries):
        try:
            print(f"\nAttempt {attempt + 1} to generate WeatherQuery...")
            response = call_hf_model(prompt, temperature=0.2)
            
            if not response:
                continue
            
            # Extract JSON from response
            json_str = extract_json_from_response(response)
            print(f"Extracted JSON: {json_str[:200]}...")
            
            # Validate with Pydantic
            weather_query = WeatherQuery.model_validate_json(json_str)
            print("✓ WeatherQuery generated and validated successfully")
            return weather_query
            
        except ValidationError as e:
            print(f"✗ Validation error on attempt {attempt + 1}:")
            for error in e.errors():
                print(f"  - {error['loc']}: {error['msg']}")
            if attempt == max_retries - 1:
                print("Max retries reached")
                return None
        except json.JSONDecodeError as e:
            print(f"✗ JSON decode error: {e}")
            if attempt == max_retries - 1:
                return None
    
    return None

# Test it
weather_query = create_weather_query(validated_request)
if weather_query:
    print(f"\nFinal WeatherQuery:\n{weather_query.model_dump_json(indent=2)}")

## Step 6: Set Up Mock Weather Databases

Create mock databases for weather data and locations to simulate real weather APIs.

In [ ]:
# Mock Weather Database
WEATHER_DATABASE = {
    "New York": {
        "current": {
            "temperature": 22.5,
            "feels_like": 24.0,
            "humidity": 65,
            "wind_speed": 8.5,
            "conditions": "Partly Cloudy",
            "timestamp": "2025-10-03T14:30:00Z"
        },
        "forecast": {
            "today": {"high": 25, "low": 18, "conditions": "Partly Cloudy"},
            "tomorrow": {"high": 23, "low": 16, "conditions": "Rainy"}
        }
    },
    "London": {
        "current": {
            "temperature": 15.2,
            "feels_like": 13.8,
            "humidity": 78,
            "wind_speed": 12.3,
            "conditions": "Overcast",
            "timestamp": "2025-10-03T14:30:00Z"
        },
        "forecast": {
            "today": {"high": 17, "low": 12, "conditions": "Overcast"},
            "tomorrow": {"high": 19, "low": 14, "conditions": "Light Rain"}
        }
    },
    "Tokyo": {
        "current": {
            "temperature": 28.1,
            "feels_like": 31.2,
            "humidity": 72,
            "wind_speed": 5.2,
            "conditions": "Sunny",
            "timestamp": "2025-10-03T14:30:00Z"
        },
        "forecast": {
            "today": {"high": 30, "low": 24, "conditions": "Sunny"},
            "tomorrow": {"high": 32, "low": 26, "conditions": "Partly Cloudy"}
        }
    }
}

# Mock Location Database for fuzzy matching
LOCATION_DATABASE = {
    "new york": ["New York, NY, USA", "New York City, NY, USA"],
    "london": ["London, England, UK", "London, Ontario, Canada"],
    "tokyo": ["Tokyo, Japan"],
    "paris": ["Paris, France", "Paris, Texas, USA"],
    "sydney": ["Sydney, NSW, Australia"],
    "los angeles": ["Los Angeles, CA, USA"],
    "chicago": ["Chicago, IL, USA"],
    "miami": ["Miami, FL, USA"]
}

# Weather Tips Database
WEATHER_TIPS = [
    {
        "condition": "rainy",
        "tip": "Don't forget an umbrella! Consider waterproof clothing for outdoor activities.",
        "keywords": ["rain", "wet", "precipitation", "storm"]
    },
    {
        "condition": "sunny",
        "tip": "Great weather for outdoor activities! Don't forget sunscreen and stay hydrated.",
        "keywords": ["sunny", "clear", "bright", "hot"]
    },
    {
        "condition": "cold",
        "tip": "Bundle up! Wear layers and protect exposed skin from the cold.",
        "keywords": ["cold", "freezing", "ice", "snow", "winter"]
    },
    {
        "condition": "windy",
        "tip": "Secure loose items and be cautious with outdoor activities. Great for flying kites!",
        "keywords": ["windy", "breezy", "gusty", "wind"]
    }
]

print("✓ Mock weather databases created")

## Step 7: Define Weather Tool Functions

These are the actual functions that will be called based on LLM decisions.

In [ ]:
def get_current_weather(args: WeatherAPIArgs) -> dict:
    """
    Get current weather data for a location.
    """
    location_key = args.location.lower().strip()
    
    # Find matching location (fuzzy matching)
    weather_data = None
    for db_location, data in WEATHER_DATABASE.items():
        if location_key in db_location.lower():
            weather_data = data
            break
    
    if not weather_data:
        return {
            "error": "Location not found", 
            "location": args.location,
            "suggestions": LOCATION_DATABASE.get(location_key, [])
        }
    
    current = weather_data["current"].copy()
    
    # Convert temperature if needed
    if args.units == "fahrenheit":
        current["temperature"] = current["temperature"] * 9/5 + 32
        if current.get("feels_like"):
            current["feels_like"] = current["feels_like"] * 9/5 + 32
    elif args.units == "kelvin":
        current["temperature"] = current["temperature"] + 273.15
        if current.get("feels_like"):
            current["feels_like"] = current["feels_like"] + 273.15
    
    return {
        "location": args.location,
        "weather_data": current,
        "units": args.units
    }

def lookup_weather_tips(args: LocationLookupArgs) -> str:
    """
    Get weather tips based on conditions or location.
    """
    query_lower = args.location_query.lower()
    
    for tip in WEATHER_TIPS:
        # Check if query matches any keywords
        if any(keyword in query_lower for keyword in tip["keywords"]):
            return f"Weather Tip: {tip['tip']}"
    
    # Default tip
    return "Weather Tip: Always check the weather forecast before heading out and dress appropriately for the conditions!"

# Test the tools
print("\n--- Testing Current Weather ---")
weather_args = WeatherAPIArgs(location="New York", query_type="current", units="celsius")
print(json.dumps(get_current_weather(weather_args), indent=2))

print("\n--- Testing Weather Tips ---")
tip_args = LocationLookupArgs(location_query="sunny day")
print(lookup_weather_tips(tip_args))

## Step 8: Weather Tool Calling with LLM

Let the LLM decide which weather tool to call based on the weather query.

In [ ]:
# Define weather tool schemas
WEATHER_TOOL_DEFINITIONS = [
    {
        "name": "get_current_weather",
        "description": "Get current weather data for a specific location",
        "parameters": WeatherAPIArgs.model_json_schema()
    },
    {
        "name": "lookup_weather_tips",
        "description": "Get weather tips and advice based on conditions or activities",
        "parameters": LocationLookupArgs.model_json_schema()
    }
]

def decide_weather_action_with_tools(weather_query: WeatherQuery, max_retries: int = 3) -> Optional[dict]:
    """
    Use LLM to decide which weather tool to call based on weather query.
    """
    tools_description = json.dumps(WEATHER_TOOL_DEFINITIONS, indent=2)
    
    prompt = f"""You are a weather information AI. Based on the weather query below, decide if you should call any tools.

Available Tools:
{tools_description}

Weather Query:
{weather_query.model_dump_json(indent=2)}

Instructions:
- If the query asks for current weather, forecast, or specific location weather, call get_current_weather
- If the query asks for advice, tips, or what to wear/do, call lookup_weather_tips
- Return ONLY a JSON object in this format:

{{
    "should_call_tool": true/false,
    "tool_name": "tool_name_here" or null,
    "tool_arguments": {{}} or null,
    "reasoning": "brief explanation"
}}

JSON Response:"""

    for attempt in range(max_retries):
        try:
            print(f"\nAttempt {attempt + 1} to decide weather tool calling...")
            response = call_hf_model(prompt, temperature=0.1)
            
            if not response:
                continue
            
            json_str = extract_json_from_response(response)
            tool_decision = json.loads(json_str)
            
            print(f"✓ Weather tool decision: {tool_decision.get('reasoning', 'No reasoning provided')}")
            return tool_decision
            
        except json.JSONDecodeError as e:
            print(f"✗ JSON decode error on attempt {attempt + 1}: {e}")
            if attempt == max_retries - 1:
                print("Max retries reached")
                return None
    
    return None

# Test weather tool decision
weather_tool_decision = decide_weather_action_with_tools(weather_query)
if weather_tool_decision:
    print(f"\nWeather Tool Decision:\n{json.dumps(weather_tool_decision, indent=2)}")

## Step 9: Execute Weather Tool Calls with Validation

In [ ]:
def execute_weather_tool_call(tool_decision: dict) -> Optional[dict]:
    """
    Execute the weather tool call with Pydantic validation of arguments.
    """
    if not tool_decision.get("should_call_tool"):
        print("No weather tool call needed")
        return None
    
    tool_name = tool_decision.get("tool_name")
    tool_args = tool_decision.get("tool_arguments")
    
    if not tool_name or not tool_args:
        print("Invalid weather tool decision format")
        return None
    
    try:
        if tool_name == "get_current_weather":
            # Validate arguments with Pydantic
            validated_args = WeatherAPIArgs.model_validate(tool_args)
            print(f"✓ Weather tool arguments validated for {tool_name}")
            result = get_current_weather(validated_args)
            return {"tool_name": tool_name, "result": result}
            
        elif tool_name == "lookup_weather_tips":
            # Validate arguments with Pydantic
            validated_args = LocationLookupArgs.model_validate(tool_args)
            print(f"✓ Weather tool arguments validated for {tool_name}")
            result = lookup_weather_tips(validated_args)
            return {"tool_name": tool_name, "result": result}
            
        else:
            print(f"Unknown weather tool: {tool_name}")
            return None
            
    except ValidationError as e:
        print(f"✗ Weather tool argument validation error:")
        for error in e.errors():
            print(f"  - {error['loc']}: {error['msg']}")
        return None

# Execute the weather tool
weather_tool_output = execute_weather_tool_call(weather_tool_decision)
if weather_tool_output:
    print(f"\nWeather Tool Output:\n{json.dumps(weather_tool_output, indent=2)}")

## Step 10: Generate Final Weather Response

In [ ]:
def generate_weather_response(
    weather_query: WeatherQuery,
    tool_output: Optional[dict] = None,
    max_retries: int = 3
) -> Optional[WeatherResponse]:
    """
    Generate final structured weather response.
    """
    schema = WeatherResponse.model_json_schema()
    
    tool_info = "No weather tools were called."
    if tool_output:
        tool_info = f"Weather tool called: {tool_output['tool_name']}\nResult: {json.dumps(tool_output['result'], indent=2)}"
    
    prompt = f"""You are a weather information AI. Create a final weather response based on all information below.

Weather Response Schema:
{json.dumps(schema, indent=2)}

Weather Query:
{weather_query.model_dump_json(indent=2)}

Weather Tool Information:
{tool_info}

Instructions:
- Return ONLY valid JSON matching the WeatherResponse schema
- Include weather_data if weather information was retrieved
- Include location_suggestions if location lookup was performed
- Choose appropriate recommended_action
- Do NOT include response_date (will be added automatically)

JSON Response:"""

    for attempt in range(max_retries):
        try:
            print(f"\nAttempt {attempt + 1} to generate weather response...")
            response = call_hf_model(prompt, temperature=0.2)
            
            if not response:
                continue
            
            json_str = extract_json_from_response(response)
            
            # Parse and validate
            response_data = json.loads(json_str)
            
            # Add response date
            response_data['response_date'] = datetime.now().isoformat()
            
            # Validate with Pydantic
            weather_response = WeatherResponse.model_validate(response_data)
            print("✓ Weather response generated and validated successfully")
            return weather_response
            
        except ValidationError as e:
            print(f"✗ Validation error on attempt {attempt + 1}:")
            for error in e.errors():
                print(f"  - {error['loc']}: {error['msg']}")
            if attempt == max_retries - 1:
                print("Max retries reached")
                return None
        except json.JSONDecodeError as e:
            print(f"✗ JSON decode error: {e}")
            if attempt == max_retries - 1:
                return None
    
    return None

# Generate final weather response
weather_response = generate_weather_response(weather_query, weather_tool_output)
if weather_response:
    print(f"\n{'='*60}")
    print("FINAL WEATHER RESPONSE")
    print('='*60)
    print(weather_response.model_dump_json(indent=2))

## Step 11: Complete End-to-End Weather Pipeline

Now let's put everything together in a single function and test with multiple weather scenarios.

In [ ]:
def process_weather_request(weather_request_json: str) -> Optional[WeatherResponse]:
    """
    Complete pipeline: validate request -> create query -> call tools -> generate response.
    """
    print("\n" + "="*60)
    print("PROCESSING WEATHER REQUEST")
    print("="*60)
    
    # Step 1: Validate input
    print("\n[1/5] Validating weather request...")
    validated_request = validate_weather_request(weather_request_json)
    if not validated_request:
        return None
    
    # Step 2: Create weather query
    print("\n[2/5] Creating weather query...")
    weather_query = create_weather_query(validated_request)
    if not weather_query:
        return None
    
    # Step 3: Decide on tool calling
    print("\n[3/5] Deciding on weather tool calls...")
    tool_decision = decide_weather_action_with_tools(weather_query)
    
    # Step 4: Execute tools if needed
    tool_output = None
    if tool_decision and tool_decision.get("should_call_tool"):
        print("\n[4/5] Executing weather tool call...")
        tool_output = execute_weather_tool_call(tool_decision)
    else:
        print("\n[4/5] No weather tool call needed")
    
    # Step 5: Generate final response
    print("\n[5/5] Generating weather response...")
    weather_response = generate_weather_response(weather_query, tool_output)
    
    return weather_response

### Test Case 1: Current Weather Query

In [ ]:
print("\n\n" + "#"*60)
print("TEST CASE 1: Current Weather Query")
print("#"*60)

test_request_1 = json.dumps({
    "name": "Jane Smith",
    "email": "jane.smith@example.com",
    "location": "New York",
    "request_type": "current"
})

response_1 = process_weather_request(test_request_1)

### Test Case 2: Weather Tips Request

In [ ]:
print("\n\n" + "#"*60)
print("TEST CASE 2: Weather Tips Request")
print("#"*60)

test_request_2 = json.dumps({
    "name": "Bob Jones",
    "email": "bob.jones@example.com",
    "location": "London",
    "request_type": "current"
})

response_2 = process_weather_request(test_request_2)

### Test Case 3: Forecast Request

In [ ]:
print("\n\n" + "#"*60)
print("TEST CASE 3: Forecast Request")
print("#"*60)

test_request_3 = json.dumps({
    "name": "Joe User",
    "email": "joe.user@example.com",
    "location": "Tokyo",
    "request_type": "forecast"
})

response_3 = process_weather_request(test_request_3)

## Step 12: Test Invalid Input (Demonstrating Pydantic Validation)

Let's see how Pydantic catches validation errors with weather requests.

### Test Case 4: Invalid Location (Too Short)

In [ ]:
print("\n\n" + "#"*60)
print("TEST CASE 4: Invalid Input (Location Too Short)")
print("#"*60)

invalid_location = json.dumps({
    "name": "Test User",
    "email": "test@example.com",
    "location": "A",  # Too short! Must be at least 2 characters
    "request_type": "current"
})

response_invalid = process_weather_request(invalid_location)

### Test Case 5: Invalid Email

In [ ]:
print("\n\n" + "#"*60)
print("TEST CASE 5: Invalid Email and Request Type")
print("#"*60)

invalid_email = json.dumps({
    "name": "Test User",
    "email": "not-an-email",  # Invalid email format
    "location": "Paris",
    "request_type": "invalid_type"  # Invalid request type
})

response_invalid_2 = process_weather_request(invalid_email)

## Summary and Key Takeaways

In [ ]:
print("\n" + "="*60)
print("SUMMARY: How Pydantic Works with Free HuggingFace Models for Weather")
print("="*60)
print("""
✓ Pydantic DOES work with free HuggingFace models for weather data!

Key Points:
1. DATA VALIDATION: Pydantic validates all weather inputs/outputs at every stage
2. STRUCTURED OUTPUTS: We use JSON schemas to guide the LLM for weather responses
3. ERROR HANDLING: Retry logic handles when models produce invalid weather JSON
4. TOOL CALLING: Pydantic validates weather API arguments before execution
5. COST: Completely free with HuggingFace Inference API

Weather Application Benefits:
- Location validation and normalization
- Temperature unit conversion validation
- Weather data structure consistency
- API argument validation for weather calls
- Structured weather responses with proper typing

Challenges with Smaller Models:
- 7B models are less consistent than GPT-4/Claude at producing valid JSON
- Require more retries and careful prompt engineering
- May need temperature tuning (lower = more consistent)
- JSON extraction from responses can be messier

Best Practices for Weather Apps:
✓ Use clear, explicit prompts with weather schema examples
✓ Implement retry logic with Pydantic validation
✓ Extract JSON carefully from weather API responses
✓ Use lower temperature (0.1-0.3) for structured weather outputs
✓ Validate weather data at every step with Pydantic models
✓ Handle location fuzzy matching with validation

Weather-Specific Validations Demonstrated:
- Location name length and format validation
- Email validation for weather alerts
- Temperature unit enum validation
- Request type validation (current, forecast, alerts)
- Weather data structure validation

Models that work well for weather (free on HuggingFace):
- mistralai/Mistral-7B-Instruct-v0.2 ✓
- meta-llama/Meta-Llama-3-8B-Instruct ✓
- microsoft/Phi-3-mini-4k-instruct ✓
- HuggingFaceH4/zephyr-7b-beta ✓
""")

## Bonus: Model Comparison Function for Weather

Test the same weather request across different models to see which performs best.

In [ ]:
def compare_weather_model_performance(weather_request_json: str, models: list[str]):
    """
    Test the same weather request across different models to see which performs best.
    """
    print("\n" + "="*60)
    print("WEATHER MODEL COMPARISON")
    print("="*60)
    
    results = {}
    
    for model_name in models:
        print(f"\n\nTesting weather request with {model_name}...")
        print("-" * 60)
        
        global MODEL_NAME
        MODEL_NAME = model_name
        
        try:
            weather_response = process_weather_request(weather_request_json)
            results[model_name] = {
                "success": weather_response is not None,
                "response": weather_response.model_dump() if weather_response else None
            }
        except Exception as e:
            results[model_name] = {
                "success": False,
                "error": str(e)
            }
    
    print("\n\n" + "="*60)
    print("WEATHER MODEL COMPARISON RESULTS")
    print("="*60)
    for model, result in results.items():
        status = "✓ SUCCESS" if result["success"] else "✗ FAILED"
        print(f"\n{model}: {status}")
    
    return results

### Run Model Comparison (Optional - Takes Time)

Uncomment and run to compare different models. This will take several minutes!

In [ ]:
# Uncomment to test multiple models for weather (this will take a while!)
# weather_test_models = [
#     "mistralai/Mistral-7B-Instruct-v0.2",
#     "microsoft/Phi-3-mini-4k-instruct",
# ]
# 
# weather_test_input = json.dumps({
#     "name": "Test User",
#     "email": "test@example.com",
#     "location": "New York", 
#     "request_type": "current"
# })
# 
# weather_comparison = compare_weather_model_performance(weather_test_input, weather_test_models)

---

## Conclusion

This tutorial demonstrates that **Pydantic absolutely works with free HuggingFace models for weather applications**, though with some caveats:

1. **Validation is rock-solid** - Pydantic catches weather data errors at every stage
2. **Smaller models require more hand-holding** - More retries, better prompts, JSON extraction
3. **The pattern is the same** - Whether using GPT-4, Claude, or Mistral-7B for weather, the Pydantic workflow is identical
4. **Cost-effective** - Everything in this weather tutorial can run on HuggingFace's free tier
5. **Weather-specific benefits** - Location validation, unit conversion, structured weather responses

The key insight remains true: **It's Pydantic all the way down** in LLM weather workflows, regardless of which model you use!